In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from datasets import load_dataset
import pandas as pd
from readability import Readability

device = 'cuda:3' # if you have a GPU

2023-03-03 02:41:18.093361: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-03 02:41:18.093381: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl").to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
steamtok = T5Tokenizer.from_pretrained('stanfordnlp/SteamSHP-flan-t5-xl')
steamshp = T5ForConditionalGeneration.from_pretrained('stanfordnlp/SteamSHP-flan-t5-xl').to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
eli5 = load_dataset("stanfordnlp/shp", data_dir="explainlikeimfive")

Using custom data configuration stanfordnlp--shp-9d14343f6fc5ff1a
Reusing dataset json (/home/prasann/.cache/huggingface/datasets/stanfordnlp___json/stanfordnlp--shp-9d14343f6fc5ff1a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
eli5['test'][20]

In [27]:
def construct_prompt(row):
    template = \
"""
The system will write a detailed and long post to respond to the user's question. Explain like the user is five years old. 

Question: """
    inp = template+row['history']+"\n Detailed Response:"
    
    
    return inp

def get_reward_single(inpdict):
    template = "POST: {context:s} \n\nRESPONSE A:{hyp:s} \n\nRESPONSE B: .\n\n Which response is better? RESPONSE "
    inp = template.format(context=inpdict['context'], hyp=inpdict['hyp'])
    x = steamtok([inp], return_tensors='pt').input_ids.to(device)
    outputs = steamshp.generate(x, return_dict_in_generate=True, output_scores=True, max_new_tokens=1)
    return torch.exp(outputs.scores[0][:, 71]) / torch.exp(outputs.scores[0][:,:]).sum(axis=1).item() # index 71 corresponds to the token for 'A'


def gen_row(rw, tok, mod, greedy=False, log=False):
    input_text = construct_prompt(rw)
    
    #print(input_text)
    input_ids = tok(input_text, return_tensors="pt").input_ids.to(device)
    if greedy:
        outputs = mod.generate(input_ids, min_new_tokens=20, max_new_tokens=200)
        outs = [tok.decode(outputs[0], skip_special_tokens=True)]
    else: 
        outputs = mod.generate(input_ids, min_new_tokens=20, max_new_tokens=200, do_sample=True, top_p=.9, temperature=.9, num_return_sequences=10)
        outs = [tok.decode(o, skip_special_tokens=True) for o in outputs]
    if log:
        print(input_text+"\n"+str(outs))
    return rw['history'], outs

def comp_rerank_greedy(row):
    inp, outs = gen_row(row, tokenizer, model, False) # do sampling for rerank
    _, greedy_out = gen_row(row, tokenizer, model, True) # do greedy 
    scos = [get_reward_single({'context':inp, 'hyp':o}) for o in outs]
    greedysco = get_reward_single({'context':inp, 'hyp':greedy_out[0]})
    # return list of stuff to put in dict 
    return pd.DataFrame(list([{'gout':greedy_out[0], 'gsco':greedysco, 'inp':inp, 'hyp':outs[i], 'sco':scos[i]} for i in range(len(scos))]))

def get_many_exouts(indf, total):
    resdfs = []
    for i in range(total):
        print(i)
        resdfs.append(comp_rerank_greedy(indf.iloc[i]))
    return pd.concat(resdfs)


In [ ]:
gen_row(eli5['test'][300], tokenizer, model, False, True)

In [23]:
eli5df = pd.DataFrame(eli5['test'])
unique_df = eli5df.drop_duplicates(subset="history")

In [31]:
eli5outs = get_many_exouts(unique_df, 100)

86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [49]:
eli5outs.to_csv("eli5_check.csv")

In [50]:
eli5outs['gsco'].mean()

0.724672908037901

In [48]:
eli5outs['gsco'] = [float(f) for f in eli5outs['gsco']]
eli5outs['sco'] = [float(f) for f in eli5outs['sco']]

In [46]:
def rer_met(rer, tgt, df, get_df=False):
    N=1
    fsort = df.sort_values(by=['gout', rer], ascending=[True, False]).groupby('gout', as_index=False).nth[:N]
    if get_df:
        return fsort
    else:
        return fsort[tgt].mean()
    
def print_data(ind, indf, outsdf):
    hist = outsdf.iloc[ind*10]['inp']
    pmpt = construct_prompt(indf[indf['history']==hist].iloc[0])
    print(pmpt)
    outs = outsdf.iloc[ind*10:(ind+1)*10]
    print("Greedy Output: \n", outs['gsco'].iloc[0], " ", outs['gout'].iloc[0], "\n")
    print("Sampling Output: \n")
    for o in range(10):
        print( outs['sco'].iloc[o], " ",outs['hyp'].iloc[o], "\n") 

In [65]:
print_data(91, unique_df, eli5outs)


The system will write a detailed and long post to respond to the user's question. Explain like the user is five years old. 

Question: Explain like I'm five years old: Why are we so concerned with automation and/or robots taking our jobs when our society has already survived exponential shifts in machinery, such as the Industrial Revolution?
 Detailed Response:
Greedy Output: 
 0.7006894946098328   The Industrial Revolution was a period of rapid technological change that radically changed the way we live. 

Sampling Output: 

0.4248104691505432   Yes, the Industrial Revolution was about mass production and mass production. If you think about the industrial revolution, it is an example of mass production. But what are we so concerned with? 

0.8918374180793762   Automation and robots taking our jobs has not been so concerning if you look at the history of technology, and especially if you look at the Industrial Revolution. It was the development of technology that produced more jobs. T

In [38]:
rer_met('sco', 'sco', eli5outs)

0.8693058776855469

In [21]:
unique_conts = list(set(hists))

In [22]:
unique_conts

["Explain like I'm five years old What keeps bed bugs in check? Why haven't they taken over the world? Do they have any natural enemies? They seem pretty unstoppable - easy breeders, can live a long time without food, can survive harsh conditions, easy hitch hikers, and they feed on an endless supply of human blood.",
 "Explain like I'm five years old: Why do some animals, like sea turtles and salmon, lay eggs away from their natural habitat? This might be a strange question, but why do sea turtles lay eggs on land and not for example dig up holes inside the ocean? They live their whole lives in the ocean, so why do they lay eggs on land? Why travel so far just to lay eggs?  Same goes for some salmon, why do they leave the oceans and lakes, and go upstream on rivers and not lay their eggs where they live?   It is probably something to do with protecting their offspring, but it seems to me that they still have predators that hunt their offspring fairly easily where they hatch/lay their 